In [ ]:
!pip install -r requirements_gpu.txt --quiet
!pip install dropbox --quiet
!pip install fastai --quiet
!pip install fastdup --quiet
!pip install imageai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 25.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import dropbox
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tempfile
import os
import shutil
import fastdup
import cv2
import numpy as np
import torch

In [ ]:
dbx_api_key = "sl.Bj4Scs7_TRE67V1ceS-wvqUtAuU1lx09ojdIYfMslE0u4iGkcunkvrhxQ1cBhzOCAcOx70c_7A8Xot6QV9VcP6baGWssfmbBki505MdXFTuAnSrQNKCA0vcQTS67jAO67WHVqublYRHP"
dbx = dropbox.Dropbox(dbx_api_key)

## Donwload relevant folders and Pre-Process the Data

In [ ]:
def read_from_dropbox(csv_path: str) -> pd.DataFrame:
    """
    Reads a CSV file from Dropbox given its path.
    """
    _, temp_file_path = tempfile.mkstemp()
    dbx.files_download_to_file(temp_file_path, csv_path)
    return pd.read_csv(temp_file_path)

def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses the given DataFrame by adding timestamps and episode_number columns.
    """
    df['timestamps'] = pd.to_datetime(df['DateTime'])
    time_diff = df['timestamps'].diff()

    threshold = 1
    start_new_episode = time_diff > pd.Timedelta(minutes=threshold)
    episode_number = start_new_episode.cumsum()
    df['episode_number'] = episode_number
    return df

# Usage:
csv_path = '/SUMMER 2023 animals only/Border Mammals Summer Results Raw Data.csv'
df = read_from_dropbox(csv_path)
df = preprocess_dataframe(df)
df.head(10)

,RootFolder,File,RelativePath,DateTime,DeleteFlag,Season,Day_Night,Mammal,Counter,Remarks,Episode,Direction,Unnamed: 12,timestamps,episode_number
0,SUMMER 2023 animals only,IMG_0022.JPG,2030,2023-07-11 18:02:22,False,Summer,Day,Bird,1,Spur-winged lapwing (sik sak),False,Pass,NaN,2023-07-11 18:02:22,0
1,SUMMER 2023 animals only,IMG_0023.JPG,2030,2023-07-11 18:02:24,False,Summer,Day,Bird,0,Spur-winged lapwing (sik sak),False,Pass,NaN,2023-07-11 18:02:24,0
2,SUMMER 2023 animals only,IMG_0025.JPG,2030,2023-07-12 00:21:20,False,Summer,Night,Jackal,1,NaN,False,In,NaN,2023-07-12 00:21:20,1
3,SUMMER 2023 animals only,IMG_0070.JPG,2030,2023-07-19 08:01:22,False,Summer,Day,Bird,1,NaN,False,Pass,NaN,2023-07-19 08:01:22,2
4,SUMMER 2023 animals only,IMG_0127.JPG,2030,2023-07-22 08:54:08,False,Summer,Day,Dog,1,NaN,False,Pass,NaN,2023-07-22 08:54:08,3
5,SUMMER 2023 animals only,IMG_0130.JPG,2030,2023-07-23 00:49:46,False,Summer,Night,Bird,1,Eurasian stone-curlew,True,Pass,NaN,2023-07-23 00:49:46,4
6,SUMMER 2023 animals only,IMG_0131.JPG,2030,2023-07-23 00:49:48,False,Summer,Night,Bird,0,Eurasian stone-curlew,True,Pass,NaN,2023-07-23 00:49:48,4
7,SUMMER 2023 animals only,IMG_0132.JPG,2030,2023-07-23 00:49:50,False,Summer,Night,Bird,0,Eurasian stone-curlew,True,Pass,NaN,2023-07-23 00:49:50,4
8,SUMMER 2023 animals only,IMG_0136.JPG,2030,2023-07-24 06:22:43,False,Summer,Day,Dog,1,NaN,True,Pass,NaN,2023-07-24 06:22:43,5
9,SUMMER 2023 animals only,IMG_0137.JPG,2030,2023-07-24 06:22:43,False,Summer,Day,Dog,0,NaN,True,Pass,NaN,2023-07-24 06:22:43,5


### Get Animals Images from DropBox

In [ ]:
!mkdir ./animal_pics

!mkdir ./animal_pics/Jackal
!mkdir ./animal_pics/Gazelle
!mkdir ./animal_pics/Dog
!mkdir ./animal_pics/Fox
!mkdir ./animal_pics/Bird
!mkdir ./animal_pics/Hare
!mkdir ./animal_pics/Cat
!mkdir ./animal_pics/Unknown_animal

local_dirs = {
    'Dog': './animal_pics/Dog',
    'Cat': './animal_pics/Cat',
    'Fox': './animal_pics/Fox',
    'Bird': './animal_pics/Bird',
    'Gazelle': './animal_pics/Gazelle',
    'Hare': './animal_pics/Hare',
    'Jackal': './animal_pics/Jackal',
    'Unknown_animal': './animal_pics/Unknown_animal',
}

In [ ]:
def get_animal(df, picture):
    return df[df['File'] == picture].iloc[0]['Mammal']

def download_animal_images(df, base_folder_path, local_dirs):
    """
    Downloads animal images from Dropbox to local directories based on animal type.

    Args:
    - df (pd.DataFrame): DataFrame containing relative paths and other necessary columns.
    - base_folder_path (str): Base folder path in Dropbox.
    - local_dirs (dict): Dictionary mapping animal names to local directory paths.
    """
    for r_path in df['RelativePath'].unique():
        print(r_path)
        folder_path = os.path.join(base_folder_path, r_path)
        filterd_df = df[df['RelativePath'] == r_path]

        # List files in the folder
        try:
            result = dbx.files_list_folder(folder_path, limit=20)
        except dropbox.exceptions.ApiError as e:
            print(f"Error listing contents of {folder_path}: {e}")
            continue

        for entry in result.entries:
            # Only process files, skip directories or other types of entries
            if not isinstance(entry, dropbox.files.FileMetadata):
                continue

            file_path = os.path.join(folder_path, entry.name)
            animal_type = get_animal(filterd_df, str(entry.name))

            # Get the local directory for this animal type
            local_dir = local_dirs.get(animal_type)
            if local_dir is not None:
                new_name = f'{r_path}_{entry.name}'
                local_path = os.path.join(local_dir, new_name)
                try:
                    dbx.files_download_to_file(local_path, file_path)
                except dropbox.exceptions.ApiError as e:
                    print(f"Error downloading {file_path}: {e}")

In [ ]:
images_path = "/SUMMER 2023 animals only/"
download_animal_images(df, images_path, local_dirs)

2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2051\leaned
Error listing contents of /SUMMER 2023 animals only/2051\leaned: ApiError('f2b8c7afc29a4d7bbeb5a0c80fecadbc', ListFolderError('path', LookupError('malformed_path', None)))
2052
2053
2054
2055
2057
2058
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070


# Load trained ImageAI model

Load the trainned model last checkpoint in order to peform detection

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection
import re
import matplotlib.patches as patches

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()

In [ ]:
detector.setModelPath("/content/drive/MyDrive/BGU Courses/data mining with miki project/yolov3_YOLO_mAP-0.36626_epoch-24.pt")
detector.setJsonPath("/content/drive/MyDrive/BGU Courses/data mining with miki project/YOLO_yolov3_detection_config.json")
detector.loadModel()

#### Functions that perform the detection pipeline

In [ ]:
def detect_objects(detector, image_path):
    detections = detector.detectObjectsFromImage(input_image=image_path, minimum_percentage_probability=50, nms_treshold=0.1)
    return detections

def print_detection_info(image_path, detections):
    image_name = os.path.basename(image_path)
    print(f"Image: {image_name}")
    for detection in detections:
        print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

def plot_image_with_boxes(image_path, detections):
    # Load the image
    image = Image.open(image_path)
    image = np.array(image)

    # Create a new figure
    plt.figure(figsize=(12,6))

    # Plot the image without detections
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
    plt.title('Original Image')
    plt.imshow(image)

    # Plot the image with detections
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
    plt.title('Image with Detections')

    # Get the current axes, creating one if necessary.
    ax = plt.gca()

    # Display the image
    ax.imshow(image)

    # Loop over all detections
    for detection in detections:
        # Get the bounding box coordinates
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = detection["box_points"]

        # Compute the width and height of the bounding box
        width = bottom_right_x - top_left_x
        height = bottom_right_y - top_left_y

        # Create the rectangle patch and add it to the plot
        rect = patches.Rectangle((top_left_x, top_left_y), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    # Show the plot
    plt.tight_layout()
    plt.show()

    # Clear the plot for future use
    plt.clf()
def detection_pipeline(detector, image_path):
    detections = detect_objects(detector, image_path)
    print_detection_info(image_path, detections)
    plot_image_with_boxes(image_path, detections)

#### Functions that perform the detection pipeline

In [ ]:
def detect_objects(detector, image_path):
    try:
        detections = detector.detectObjectsFromImage(input_image=image_path, minimum_percentage_probability=50, nms_treshold=0.1)
        return detections
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return []

def print_detection_info(image_path, detections):
    image_name = os.path.basename(image_path)
    print(f"Image: {image_name}")
    for detection in detections:
        print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

def plot_image_with_boxes(image_path, detections):
    # Load the image
    image = Image.open(image_path)
    image = np.array(image)

    # Create a new figure
    plt.figure(figsize=(12,6))

    # Plot the image without detections
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
    plt.title('Original Image')
    plt.imshow(image)

    # Plot the image with detections
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
    plt.title('Image with Detections')

    # Get the current axes, creating one if necessary.
    ax = plt.gca()

    # Display the image
    ax.imshow(image)

    # Loop over all detections
    for detection in detections:
        # Get the bounding box coordinates
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = detection["box_points"]

        # Compute the width and height of the bounding box
        width = bottom_right_x - top_left_x
        height = bottom_right_y - top_left_y

        # Create the rectangle patch and add it to the plot
        rect = patches.Rectangle((top_left_x, top_left_y), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    # Show the plot
    plt.tight_layout()
    plt.show()

    # Clear the plot for future use
    plt.clf()
def detection_pipeline(detector, image_path):
    detections = detect_objects(detector, image_path)
    print_detection_info(image_path, detections)
    plot_image_with_boxes(image_path, detections)

### Evaluate Model Detection

In [ ]:
def precision(tp, fp):
    """Calculate the precision based on the true and false positives."""
    try:
        return tp / (tp + fp)
    except ZeroDivisionError:
        return 0

def recall(tp, fn):
    """Calculate the recall based on the true positives and false negatives."""
    try:
        return tp / (tp + fn)
    except ZeroDivisionError:
        return 0

def f1_score(precision, recall):
    try:
        return 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        return 0

Eval dir of images

In [ ]:
def evaluate_detector_on_directory(detector, directory_path, ground_truth_df, plot_results=False, num_samples=None):
    image_files = os.listdir(directory_path)

    # Initialize counters for true positives, false positives, false negatives, and evaluated samples
    tp = 0
    fp = 0
    fn = 0
    samples_evaluated = 0

    for image_file in image_files:
        # If we've evaluated enough samples, break the loop
        if num_samples is not None and samples_evaluated >= num_samples:
            break

        image_path = os.path.join(directory_path, image_file)
        detections = detect_objects(detector, image_path)
        if plot_results:
            detection_pipeline(detector, image_path)

        # Extract the image name from the image path
        image_name = os.path.basename(image_path).split('_', 1)[1]

        # Find the ground truth data for this image
        ground_truth = ground_truth_df[ground_truth_df['File'] == image_name]

        # Check if there is ground truth data for this image
        if ground_truth.empty:
            print(f"No ground truth data found for image {image_name}")
            continue

        # Get the true labels and predicted labels for each image
        true_animal = ground_truth['Mammal'].values[0]
        pred_animal = detections[0]['name'] if len(detections) > 0 else 'No Detection'

        # Compare the true and predicted labels to update the counters
        if true_animal == pred_animal:
            tp += 1
        else:
            if pred_animal == 'No Detection':
                fn += 1
            else:
                fp += 1

        # Increment the counter for evaluated samples
        samples_evaluated += 1

    return tp, fp, fn, samples_evaluated

    # Calculate precision and recall
    precision_value = precision(tp, fp)
    recall_value = recall(tp, fn)

    # print(f'Precision: {precision_value}')
    # print(f'Recall: {recall_value}')

def evaluate_on_all_images(detector, base_path, ground_truth_df, plot_results=False, num_samples=None):
    animal_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

    total_tp = 0
    total_fp = 0
    total_fn = 0
    total_samples = 0

    for animal_dir in animal_dirs:
        directory_path = os.path.join(base_path, animal_dir)
        tp, fp, fn, samples_evaluated = evaluate_detector_on_directory(detector, directory_path, ground_truth_df, plot_results, num_samples)

        total_tp += tp
        total_fp += fp
        total_fn += fn
        total_samples += samples_evaluated

    # Calculate precision, recall, accuracy and F1 score
    overall_precision = precision(total_tp, total_fp)
    overall_recall = recall(total_tp, total_fn)
    overall_accuracy = total_tp / total_samples
    overall_f1 = f1_score(overall_precision, overall_recall)

    print(f'Accuracy: {overall_accuracy}')
    print(f'Precision: {overall_precision}')
    print(f'Recall: {overall_recall}')
    print(f'F1 Score: {overall_f1}')

# Invoke the function
evaluate_on_all_images(detector, "/content/animal_pics", df, False)

Error processing /content/animal_pics/Jackal/2035_IMAG0004.MOV: local variable 'img' referenced before assignment
Accuracy: 0.23416506717850288
Precision: 0.40131578947368424
Recall: 0.35988200589970504
F1 Score: 0.3794712286158632


##### Evaluate Dog directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Dog", df, False)

Precision: 0.6666666666666666
Recall: 0.7619047619047619


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Dog", df, True, 20)

##### Evaluate Fox directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Fox", df, False, 50)

Precision: 0.24242424242424243
Recall: 0.32


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Fox", df, True, 20)

##### Evaluate Jackal directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Jackal", df, False, 50)

Precision: 0.5714285714285714
Recall: 0.3333333333333333


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Jackal", df, True, 20)

##### Evaluate Gazelle directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Gazelle", df, False, 50)

Precision: 0.46875
Recall: 0.45454545454545453


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Gazelle", df, True, 20)

##### Evaluate Cat directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Cat", df, False, 50)

Precision: 0
Recall: 0


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Cat", df, True, 20)

Precision: 0
Recall: 0


##### Evaluate Bird directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Bird", df, False, 50)

Precision: 0.21428571428571427
Recall: 0.15789473684210525


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Bird", df, True, 20)

##### Evaluate Hare directory from Winter data

In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Hare", df, False, 50)

Precision: 0.25925925925925924
Recall: 0.4375


In [ ]:
evaluate_detector_on_directory(detector, "/content/animal_pics/Hare", df, True, 20)